# Module 03: Exploring NIXL (NVIDIA Inference Xfer Library)

> **Goal**: Understand how NIXL enables fast GPU-to-GPU KV cache transfers in disaggregated serving.

---

## What is NIXL?

**NIXL** (NVIDIA Inference Xfer Library) is a high-performance data transfer library designed for AI inference. It enables **direct GPU-to-GPU transfers** without CPU involvement using RDMA (Remote Direct Memory Access).

### Why NIXL Matters for Disaggregated Serving

In disaggregated LLM serving, prefill and decode run on separate GPUs. The KV cache must move between them:

```
┌─────────────────┐                    ┌─────────────────┐
│  Prefill Worker │                    │  Decode Worker  │
│     (GPU 0)     │                    │     (GPU 1)     │
│                 │                    │                 │
│  KV Cache       │═══════NIXL════════▶│  KV Cache       │
│  (VRAM)         │   Direct Transfer  │  (VRAM)         │
│                 │   ~10-50 GB/s      │                 │
└─────────────────┘                    └─────────────────┘
```

**Without NIXL**, you'd need: GPU 0 → CPU RAM → Network/IPC → CPU RAM → GPU 1

**With NIXL**: GPU 0 → GPU 1 (direct RDMA, bypasses CPU entirely)

---

## Step 1: Install and Verify NIXL

In [26]:
# Install NIXL
!uv pip install nixl

import nixl
print(f"NIXL installed at: {nixl.__file__}")
!uv pip show nixl | grep -E "^(Name|Version)"

Using Python 3.12.3 environment at: /root/src/github.com/sara4dev/ai-dynamo-the-hard-way/.venv
Audited 1 package in 23ms
NIXL installed at: /root/src/github.com/sara4dev/ai-dynamo-the-hard-way/.venv/lib/python3.12/site-packages/nixl/__init__.py
Using Python 3.12.3 environment at: /root/src/github.com/sara4dev/ai-dynamo-the-hard-way/.venv
Name: nixl
Version: 0.8.0


---

## Step 2: NIXL Core Concepts

Before diving into the transfer demo, let's understand the key NIXL components.

### Key Components

| Component | Purpose |
|-----------|--------|
| **nixlAgent** | Core class - each process creates one to participate in transfers |
| **Backend (UCX)** | Handles actual data movement via RDMA, NVLink, or TCP |
| **Memory Registration** | Makes GPU memory accessible for RDMA operations |
| **Metadata Exchange** | Allows agents to discover each other's registered memory |
| **Transfer Requests** | Describes what data to move and where |

### Memory Types

| Type | Description |
|------|-------------|
| `VRAM_SEG` | GPU memory (what we'll use for KV cache) |
| `DRAM_SEG` | CPU memory |
| `FILE_SEG` | File storage |

### Transfer Operations

| Operation | Description |
|-----------|-------------|
| `NIXL_READ` | Pull data FROM remote agent |
| `NIXL_WRITE` | Push data TO remote agent |

### Key Status Codes

| Status | Meaning |
|--------|---------|
| `NIXL_SUCCESS` | Operation completed |
| `NIXL_IN_PROG` | Transfer in progress (async) |

---

## Step 3: Multi-Process GPU-to-GPU Transfer Demo

Now let's perform an actual GPU-to-GPU transfer using NIXL with **separate processes** - exactly how Dynamo runs prefill and decode workers in production.

### Architecture

Run the cell below to see the sequence diagram:

In [27]:
import base64
import urllib.parse
from IPython.display import display, Image, SVG

# Mermaid diagram definition
mermaid_code = """
sequenceDiagram
    participant P as Prefill (GPU 0)
    participant E as etcd
    participant D as Decode (GPU 1)

    Note over P: Create nixl_agent (listener)<br/>Allocate KV cache<br/>Register memory with NIXL
    Note over D: Create nixl_agent<br/>Allocate dest buffer<br/>Register memory with NIXL
    
    P->>E: Register endpoint (/nixl/{id}/prefill_endpoint)
    D->>E: Poll for prefill endpoint
    E-->>D: Return prefill endpoint
    
    D->>P: Connect to prefill
    P->>D: Exchange NIXL metadata
    D->>P: Exchange NIXL metadata
    P->>D: Send transfer descriptors
    
    Note over D: Create READ transfer
    P-->>D: DATA TRANSFER (GPU 0 → GPU 1)
    Note over D: Verify data
    
    D->>E: Signal done (/nixl/{id}/done)
    E-->>P: Receive done signal
    
    Note over P,D: Cleanup
"""

# Render using mermaid.ink service (generates SVG from mermaid code)
encoded = base64.urlsafe_b64encode(mermaid_code.encode('utf-8')).decode('ascii')
img_url = f"https://mermaid.ink/img/{encoded}"

# Display the diagram
display(Image(url=img_url, width=800))
print("✓ Architecture diagram rendered (via mermaid.ink)")

✓ Architecture diagram rendered (via mermaid.ink)


### How This Maps to Dynamo

| This Demo | Dynamo Production |
|-----------|-------------------|
| etcd for endpoint discovery | etcd for service registry |
| etcd for completion signal | NATS for request coordination |
| NIXL for data transfer | NIXL for KV cache transfer |
| Direct IP:port connection | Kubernetes service discovery |

### Transport Selection

UCX automatically selects the best available transport:

| Transport | Used When | Bandwidth |
|-----------|-----------|-----------|
| NVLink | Same node, NVLink available | ~600 GB/s |
| cuda_ipc | Same node, P2P enabled | ~25-50 GB/s |
| RDMA | Cross-node, RDMA NIC | ~25-100 GB/s |
| TCP (fallback) | No P2P, no RDMA | ~1-10 Gbps |

> **Note**: Consumer GPUs (like RTX) often don't support P2P over PCIe. NIXL will automatically fall back to TCP + CPU staging. Datacenter GPUs (A100, H100) have NVLink and full RDMA support.

In [28]:
import torch
import os
import time

# Install etcd client (using etcd3gw for protobuf compatibility)
!uv pip install etcd3gw -q
from etcd3gw import Etcd3Client

print("=" * 70)
print("NIXL + etcd MULTI-PROCESS GPU-TO-GPU TRANSFER")
print("=" * 70)

# Verify we have 2 GPUs
assert torch.cuda.device_count() >= 2, "This demo requires 2 GPUs"
print(f"\n✓ Found {torch.cuda.device_count()} GPUs:")
for i in range(2):
    print(f"   GPU {i}: {torch.cuda.get_device_name(i)}")

# Check P2P capability (affects which transport UCX will use)
p2p_0_to_1 = torch.cuda.can_device_access_peer(0, 1)
p2p_1_to_0 = torch.cuda.can_device_access_peer(1, 0)
print(f"\n✓ P2P Access Check:")
print(f"   GPU 0 → GPU 1: {p2p_0_to_1}")
print(f"   GPU 1 → GPU 0: {p2p_1_to_0}")

if not (p2p_0_to_1 and p2p_1_to_0):
    print("\n⚠️  P2P not available - NIXL will use TCP + CPU staging (slower)")
    print("   This is normal for consumer GPUs (RTX series)")
    print("   Datacenter GPUs (A100, H100) have NVLink with full P2P support")
else:
    print("\n✓ P2P enabled - NIXL can use cuda_ipc for fast transfers")

# Connect to etcd (used for service discovery, like Dynamo)
ETCD_HOST = os.environ.get("ETCD_HOST", "localhost")
ETCD_PORT = int(os.environ.get("ETCD_PORT", 2379))
try:
    etcd_client = Etcd3Client(host=ETCD_HOST, port=ETCD_PORT)
    etcd_client.status()
    print(f"\n✓ Connected to etcd at {ETCD_HOST}:{ETCD_PORT}")
    print("  Dynamo uses etcd for service discovery and coordination")
except Exception as e:
    raise RuntimeError(
        f"etcd not available at {ETCD_HOST}:{ETCD_PORT}. "
        "Start with: docker run -d -p 2379:2379 quay.io/coreos/etcd:v3.5.0 "
        "/usr/local/bin/etcd --listen-client-urls http://0.0.0.0:2379 "
        "--advertise-client-urls http://0.0.0.0:2379"
    ) from e

# Configuration for the demo
PREFILL_PORT = 15555  # Port for prefill worker to listen on
print(f"\n✓ Prefill worker will listen on port {PREFILL_PORT}")

NIXL + etcd MULTI-PROCESS GPU-TO-GPU TRANSFER

✓ Found 2 GPUs:
   GPU 0: NVIDIA GeForce RTX 5090
   GPU 1: NVIDIA GeForce RTX 5090

✓ P2P Access Check:
   GPU 0 → GPU 1: False
   GPU 1 → GPU 0: False

⚠️  P2P not available - NIXL will use TCP + CPU staging (slower)
   This is normal for consumer GPUs (RTX series)
   Datacenter GPUs (A100, H100) have NVLink with full P2P support

✓ Connected to etcd at localhost:2379
  Dynamo uses etcd for service discovery and coordination

✓ Prefill worker will listen on port 15555


### Step 3.1: Define the Prefill Worker

The prefill worker runs on GPU 0 as a **separate process**. It:
1. Creates a NIXL agent with a **listener thread** (accepts connections)
2. Allocates and registers KV cache memory on GPU 0
3. **Registers its endpoint in etcd** (so decode can discover it)
4. Waits for decode to connect and exchange NIXL metadata
5. Sends transfer descriptors so decode knows the memory layout
6. **Waits for completion signal in etcd**

In [29]:
def prefill_worker(request_id: str, etcd_host: str, etcd_port: int, nixl_port: int):
    """
    Prefill Worker - runs in a separate process on GPU 0
    
    In production (Dynamo), this would be a separate container/pod
    running on a prefill-optimized GPU instance.
    
    Uses etcd for:
    - Service discovery (registers endpoint so decode can find us)
    - Completion signaling (waits for decode to signal done)
    
    Uses NIXL for:
    - Memory registration
    - Metadata exchange
    - Data transfer
    """
    import os
    import time
    import torch
    from etcd3gw import Etcd3Client
    from nixl._api import nixl_agent, nixl_agent_config
    
    # etcd keys for this request (like Dynamo uses request IDs)
    endpoint_key = f"/nixl/{request_id}/prefill_endpoint"
    done_key = f"/nixl/{request_id}/done"
    result_key = f"/nixl/{request_id}/prefill_result"
    
    print(f"[PREFILL] Starting worker (PID: {os.getpid()}) on port {nixl_port}")
    
    try:
        etcd = Etcd3Client(host=etcd_host, port=etcd_port)
        
        # === STEP 1: Create NIXL agent with listener ===
        config = nixl_agent_config(
            enable_prog_thread=True,
            enable_listen_thread=True,  # Accept incoming connections
            listen_port=nixl_port,      # Listen on this port
            backends=["UCX"]
        )
        agent = nixl_agent("prefill-worker", config)
        print(f"[PREFILL] Created agent, listening on port {nixl_port}")
        
        # === STEP 2: Allocate KV cache on GPU 0 ===
        # Use random data like production (not deterministic arange)
        # Use a large buffer (1 GB) to properly measure TCP bandwidth
        torch.cuda.set_device(0)
        torch.manual_seed(42)  # For reproducibility in demo
        kv_cache = torch.randn(16384, 32768, device='cuda:0', dtype=torch.float16)  # 1 GB
        size_bytes = kv_cache.numel() * kv_cache.element_size()
        size_gb = size_bytes / (1024 ** 3)
        flat = kv_cache.flatten()
        print(f"[PREFILL] KV cache on GPU 0: {tuple(kv_cache.shape)}, {size_gb:.2f} GB")
        print(f"[PREFILL] SOURCE tensor (first 5): [{flat[0]:.4f}, {flat[1]:.4f}, {flat[2]:.4f}, {flat[3]:.4f}, {flat[4]:.4f}]")
        print(f"[PREFILL] SOURCE tensor (last 5):  [{flat[-5]:.4f}, {flat[-4]:.4f}, {flat[-3]:.4f}, {flat[-2]:.4f}, {flat[-1]:.4f}]")
        
        # === STEP 3: Register memory with NIXL ===
        reg_descs = agent.register_memory(kv_cache)
        print(f"[PREFILL] Memory registered with NIXL")
        
        # Build transfer descriptors (decode will need this)
        xfer_descs = agent.get_xfer_descs(kv_cache)
        xfer_descs_serialized = agent.get_serialized_descs(xfer_descs)
        
        # === STEP 4: Register endpoint in etcd (service discovery) ===
        # In Dynamo, workers register themselves so others can discover them
        endpoint_info = f"127.0.0.1:{nixl_port}"
        etcd.put(endpoint_key, endpoint_info)
        print(f"[PREFILL] Registered endpoint in etcd: {endpoint_key} = {endpoint_info}")
        print(f"[PREFILL] Decode worker can now discover us via etcd")
        
        # === STEP 5: Wait for decode to connect ===
        print(f"[PREFILL] Waiting for decode worker to connect...")
        while not agent.check_remote_metadata("decode-worker"):
            time.sleep(0.1)
        print(f"[PREFILL] Decode worker connected!")
        
        # === STEP 6: Send transfer descriptors to decode ===
        agent.send_notif("decode-worker", xfer_descs_serialized)
        print(f"[PREFILL] Sent transfer descriptors to decode worker")
        
        # === STEP 7: Wait for completion signal in etcd ===
        print(f"[PREFILL] Waiting for completion signal in etcd...")
        while True:
            result = etcd.get(done_key)
            if result and result[0]:
                print(f"[PREFILL] Received completion signal from etcd")
                break
            time.sleep(0.1)
        
        # === STEP 8: Cleanup ===
        agent.deregister_memory(reg_descs)
        etcd.put(result_key, "success")
        print(f"[PREFILL] Cleanup complete")
        
    except Exception as e:
        import traceback
        print(f"[PREFILL] Error: {e}")
        traceback.print_exc()
        try:
            etcd.put(result_key, f"error: {e}")
        except:
            pass

print("✓ Prefill worker function defined")

✓ Prefill worker function defined


### Step 3.2: Define the Decode Worker

The decode worker runs on GPU 1 as a **separate process**. It:
1. **Discovers prefill's endpoint from etcd** (service discovery)
2. Creates a NIXL agent and connects to prefill
3. Allocates destination buffer on GPU 1
4. Exchanges NIXL metadata with prefill
5. Creates a `READ` transfer request to **pull** data from prefill
6. Executes the transfer and verifies data integrity
7. **Signals completion via etcd**

In [30]:
def decode_worker(request_id: str, etcd_host: str, etcd_port: int):
    """
    Decode Worker - runs in a separate process on GPU 1
    
    In production (Dynamo), this would be a separate container/pod
    running on a decode-optimized GPU instance.
    
    Uses etcd for:
    - Service discovery (finds prefill's endpoint)
    - Completion signaling (notifies prefill when done)
    
    Uses NIXL for:
    - Memory registration
    - Metadata exchange
    - Data transfer
    """
    import os
    import time
    import torch
    from etcd3gw import Etcd3Client
    from nixl._api import nixl_agent, nixl_agent_config
    
    # etcd keys for this request
    endpoint_key = f"/nixl/{request_id}/prefill_endpoint"
    done_key = f"/nixl/{request_id}/done"
    result_key = f"/nixl/{request_id}/decode_result"
    
    print(f"[DECODE] Starting worker (PID: {os.getpid()})")
    
    try:
        etcd = Etcd3Client(host=etcd_host, port=etcd_port)
        
        # === STEP 1: Discover prefill endpoint from etcd ===
        print(f"[DECODE] Discovering prefill endpoint from etcd...")
        prefill_endpoint = None
        for _ in range(60):  # Timeout after 60 seconds
            result = etcd.get(endpoint_key)
            if result and result[0]:
                value = result[0]
                prefill_endpoint = value.decode('utf-8') if isinstance(value, bytes) else value
                break
            time.sleep(1)
        
        if not prefill_endpoint:
            raise TimeoutError(f"Prefill endpoint not found in etcd at {endpoint_key}")
        
        prefill_ip, prefill_port = prefill_endpoint.split(":")
        prefill_port = int(prefill_port)
        print(f"[DECODE] Found prefill at {prefill_ip}:{prefill_port} (from etcd)")
        
        # === STEP 2: Create NIXL agent ===
        config = nixl_agent_config(
            enable_prog_thread=True,
            enable_listen_thread=True,
            listen_port=0,  # Use any available port
            backends=["UCX"]
        )
        agent = nixl_agent("decode-worker", config)
        print(f"[DECODE] Agent created")
        
        # === STEP 3: Allocate destination buffer on GPU 1 ===
        # Match prefill size (1 GB)
        torch.cuda.set_device(1)
        dest_buffer = torch.zeros(16384, 32768, device='cuda:1', dtype=torch.float16)  # 1 GB
        size_bytes = dest_buffer.numel() * dest_buffer.element_size()
        size_gb = size_bytes / (1024 ** 3)
        flat = dest_buffer.flatten()
        print(f"[DECODE] Dest buffer on GPU 1: {tuple(dest_buffer.shape)}, {size_gb:.2f} GB")
        print(f"[DECODE] BEFORE transfer (first 5): [{flat[0]:.4f}, {flat[1]:.4f}, {flat[2]:.4f}, {flat[3]:.4f}, {flat[4]:.4f}]")
        print(f"[DECODE] BEFORE transfer (last 5):  [{flat[-5]:.4f}, {flat[-4]:.4f}, {flat[-3]:.4f}, {flat[-2]:.4f}, {flat[-1]:.4f}]")
        
        # === STEP 4: Register local memory ===
        reg_descs = agent.register_memory(dest_buffer)
        print(f"[DECODE] Memory registered with NIXL")
        
        # === STEP 5: Exchange NIXL metadata with prefill ===
        agent.send_local_metadata(prefill_ip, prefill_port)
        agent.fetch_remote_metadata("prefill-worker", prefill_ip, prefill_port)
        print(f"[DECODE] NIXL metadata exchange initiated")
        
        while not agent.check_remote_metadata("prefill-worker"):
            time.sleep(0.1)
        print(f"[DECODE] Have prefill's NIXL metadata")
        
        # === STEP 6: Receive transfer descriptors from prefill ===
        remote_descs = None
        while remote_descs is None:
            notifs = agent.get_new_notifs()
            if "prefill-worker" in notifs and notifs["prefill-worker"]:
                remote_descs = agent.deserialize_descs(notifs["prefill-worker"][0])
                break
            time.sleep(0.1)
        print(f"[DECODE] Received prefill's transfer descriptors")
        
        local_descs = agent.get_xfer_descs(dest_buffer)
        
        # === STEP 7: Create transfer request ===
        print(f"[DECODE] Creating READ transfer request...")
        xfer_handle = agent.initialize_xfer(
            "READ",           # Pull from remote
            local_descs,      # Where to put data (GPU 1)
            remote_descs,     # Where to get data (GPU 0)
            "prefill-worker"  # Remote agent name
        )
        print(f"[DECODE] Transfer request created")
        
        # === STEP 8: Execute the transfer ===
        print(f"[DECODE] Executing transfer GPU 0 → GPU 1...")
        start_time = time.perf_counter()
        
        status = agent.transfer(xfer_handle)
        print(f"[DECODE] Transfer posted, status: {status}")
        
        while True:
            state = agent.check_xfer_state(xfer_handle)
            if state == "DONE":
                break
            elif state == "ERR":
                raise RuntimeError("Transfer failed")
            time.sleep(0.001)
        
        elapsed = time.perf_counter() - start_time
        bandwidth_gbps = (size_bytes * 8 / 1e9) / elapsed  # Gbps (network convention)
        bandwidth_gbs = (size_bytes / (1024 ** 3)) / elapsed  # GB/s (for reference)
        
        print(f"[DECODE] Transfer complete!")
        print(f"          Size: {size_gb:.2f} GB")
        print(f"          Time: {elapsed * 1000:.1f} ms")
        print(f"          Bandwidth: {bandwidth_gbps:.2f} Gbps ({bandwidth_gbs:.2f} GB/s)")
        
        # === STEP 9: Print received data for manual inspection ===
        torch.cuda.synchronize(1)
        received = dest_buffer.flatten()
        
        print(f"[DECODE] AFTER transfer (first 5):  [{received[0]:.4f}, {received[1]:.4f}, {received[2]:.4f}, {received[3]:.4f}, {received[4]:.4f}]")
        print(f"[DECODE] AFTER transfer (last 5):   [{received[-5]:.4f}, {received[-4]:.4f}, {received[-3]:.4f}, {received[-2]:.4f}, {received[-1]:.4f}]")
        print(f"[DECODE] ✓ Transfer complete - compare with PREFILL SOURCE values above to verify!")
        etcd.put(result_key, "success")
        
        # === STEP 10: Signal completion via etcd ===
        etcd.put(done_key, "true")
        print(f"[DECODE] Signaled completion via etcd")
        
        # === STEP 11: Cleanup ===
        agent.release_xfer_handle(xfer_handle)
        agent.remove_remote_agent("prefill-worker")
        agent.deregister_memory(reg_descs)
        print(f"[DECODE] Cleanup complete")
        
    except Exception as e:
        import traceback
        print(f"[DECODE] Error: {e}")
        traceback.print_exc()
        try:
            etcd.put(result_key, f"error: {e}")
            etcd.put(done_key, "true")  # Signal done so prefill doesn't hang
        except:
            pass

print("✓ Decode worker function defined")

✓ Decode worker function defined


### Step 3.3: Run the Multi-Process Transfer

Now we spawn both workers as separate processes and watch them communicate via NIXL + etcd.

**Key points:**
- Each worker runs in its own **separate process** (different PID)
- Prefill registers its endpoint in **etcd** (service discovery)
- Decode discovers prefill via **etcd**, then connects directly
- NIXL metadata is exchanged via **direct network connection**
- Data transfer uses **UCX** (with best available transport)
- Completion is signaled via **etcd** (like NATS in Dynamo)

In [31]:
import uuid

# Use 'multiprocess' instead of 'multiprocessing' - it uses dill instead of pickle
# and can serialize functions defined in notebooks. Drop-in replacement.
!uv pip install multiprocess -q
import multiprocess as mp

# Set multiprocessing start method for CUDA compatibility
try:
    mp.set_start_method('spawn', force=True)
except RuntimeError:
    pass  # Already set

print("=" * 70)
print("EXECUTING MULTI-PROCESS TRANSFER")
print("=" * 70)

# Create unique request ID (like Dynamo uses for each inference request)
request_id = str(uuid.uuid4())[:8]
print(f"\n📦 Request ID: {request_id}")
print(f"\n   etcd keys that will be used:")
print(f"     /nixl/{request_id}/prefill_endpoint  - Service discovery")
print(f"     /nixl/{request_id}/done              - Completion signal")
print(f"     /nixl/{request_id}/prefill_result    - Prefill status")
print(f"     /nixl/{request_id}/decode_result     - Decode status")
print()

prefill_proc = None
decode_proc = None

try:
    # Spawn worker processes
    prefill_proc = mp.Process(
        target=prefill_worker,
        args=(request_id, ETCD_HOST, ETCD_PORT, PREFILL_PORT)
    )
    decode_proc = mp.Process(
        target=decode_worker,
        args=(request_id, ETCD_HOST, ETCD_PORT)
    )
    
    print("─" * 70)
    print("Starting worker processes...")
    print("─" * 70)
    
    # Start both processes
    prefill_proc.start()
    decode_proc.start()
    
    # Wait for completion (timeout after 60 seconds)
    prefill_proc.join(timeout=60)
    decode_proc.join(timeout=60)
    
    print()
    print("─" * 70)
    print("Results (from etcd):")
    print("─" * 70)
    
    # Collect results from etcd
    def get_etcd_value(key):
        result = etcd_client.get(key)
        if result and result[0]:
            value = result[0]
            return value.decode('utf-8') if isinstance(value, bytes) else value
        return "not reported"
    
    prefill_status = get_etcd_value(f"/nixl/{request_id}/prefill_result")
    decode_status = get_etcd_value(f"/nixl/{request_id}/decode_result")
    
    print(f"  Prefill: {prefill_status}")
    print(f"  Decode:  {decode_status}")
    
    if prefill_status == "success" and decode_status == "success":
        print()
        print("🎉 SUCCESS! Multi-process GPU-to-GPU transfer completed!")
        print()
        print("What happened:")
        print("  1. Prefill registered its endpoint in etcd (service discovery)")
        print("  2. Decode discovered prefill via etcd")
        print("  3. They connected directly and exchanged NIXL metadata")
        print("  4. Decode pulled the KV cache from GPU 0 to GPU 1")
        print("  5. Decode signaled completion via etcd")
        print()
        print("This is exactly how Dynamo coordinates workers!")
    else:
        print()
        print("⚠️ Some workers reported issues - check output above")

finally:
    # Cleanup etcd keys
    for suffix in ["/prefill_endpoint", "/done", "/prefill_result", "/decode_result"]:
        try:
            etcd_client.delete(f"/nixl/{request_id}{suffix}")
        except:
            pass
    
    # Terminate any hanging processes
    if prefill_proc and prefill_proc.is_alive():
        prefill_proc.terminate()
    if decode_proc and decode_proc.is_alive():
        decode_proc.terminate()

print()
print("=" * 70)
print("DEMO COMPLETE")
print("=" * 70)

EXECUTING MULTI-PROCESS TRANSFER

📦 Request ID: ca751042

   etcd keys that will be used:
     /nixl/ca751042/prefill_endpoint  - Service discovery
     /nixl/ca751042/done              - Completion signal
     /nixl/ca751042/prefill_result    - Prefill status
     /nixl/ca751042/decode_result     - Decode status

──────────────────────────────────────────────────────────────────────
Starting worker processes...
──────────────────────────────────────────────────────────────────────
[PREFILL] Starting worker (PID: 2969385) on port 15555
[DECODE] Starting worker (PID: 2969386)
[DECODE] Discovering prefill endpoint from etcd...
2026-02-01 22:40:44 NIXL INFO    _api.py:363 Backend UCX was instantiated
2026-02-01 22:40:44 NIXL INFO    _api.py:253 Initialized NIXL agent: prefill-worker
[PREFILL] Created agent, listening on port 15555
[PREFILL] KV cache on GPU 0: (16384, 32768), 1.00 GB
[PREFILL] SOURCE tensor (first 5): [0.1940, 2.1621, -0.1720, 0.8491, -1.9248]
[PREFILL] SOURCE tensor (last

---

## Step 4: How This Maps to Dynamo

Here's how our demo maps to real-world disaggregated LLM inference in Dynamo:

| Demo Component | Dynamo Equivalent |
|----------------|-------------------|
| `prefill_worker()` | Prefill Worker container |
| `decode_worker()` | Decode Worker container |
| etcd endpoint registration | etcd service registry |
| etcd endpoint discovery | Kubernetes / etcd service discovery |
| etcd completion signal | NATS request coordination |
| `nixl_agent` | NIXL agent in each worker |
| `send_local_metadata()` | Direct NIXL metadata exchange |
| `initialize_xfer("READ", ...)` | KV cache transfer request |
| UCX backend | RDMA/NVLink for fast transfers |

### Dynamo Integration

In Dynamo, NIXL is enabled with:
```bash
dynamo run ... --disaggregation-transfer-backend nixl
```

Dynamo handles all the agent creation, metadata exchange, and transfer coordination automatically.

### Production Pattern (Simplified)

```python
# Prefill worker (after generating KV cache)
from nixl._api import nixl_agent, nixl_agent_config

agent = nixl_agent("prefill", nixl_agent_config(enable_listen_thread=True, listen_port=5555))
reg = agent.register_memory(kv_cache)
# Register endpoint in etcd for service discovery
etcd.put(f"/workers/prefill/{worker_id}", f"{ip}:5555")

# Decode worker (before starting decode)
# Discover prefill via etcd
prefill_endpoint = etcd.get(f"/workers/prefill/{target_worker}")
prefill_ip, prefill_port = prefill_endpoint.split(":")

agent = nixl_agent("decode", nixl_agent_config())
agent.fetch_remote_metadata("prefill", prefill_ip, int(prefill_port))
xfer = agent.initialize_xfer("READ", local_descs, remote_descs, "prefill")
agent.transfer(xfer)
# ... poll check_xfer_state() until "DONE", then use KV cache
```

### Transport Selection on Production Hardware

| Hardware | Transport | Bandwidth |
|----------|-----------|-----------|
| DGX/HGX with NVLink | NVLink via UCX | 600+ GB/s |
| Cross-node with RDMA NIC | RDMA via UCX | 25-100 GB/s |
| Consumer GPUs (RTX) | TCP + CPU staging | 1-10 Gbps |

---

## Key Takeaways

1. **NIXL + etcd** - Dynamo uses etcd for service discovery and NIXL for high-performance data transfers

2. **Service discovery** - Workers register endpoints in etcd so others can find them

3. **`nixl_agent`** is the core class - each worker creates one to participate in transfers

4. **Listener threads** (`enable_listen_thread=True`) allow agents to accept incoming connections

5. **Network-based connections** - agents connect via IP:port discovered from etcd

6. **UCX backend** automatically selects the best transport (NVLink > cuda_ipc > RDMA > TCP)

7. **Memory registration** is required before transfers (enables RDMA/DMA mappings)

8. **Pull model** - receivers create `READ` transfer requests to pull data from senders

9. **Async execution** - `transfer()` is non-blocking, poll with `check_xfer_state()`

10. **Fallback support** - on systems without P2P/NVLink, NIXL falls back to TCP + CPU staging

---

### Resources

| Resource | URL |
|----------|-----|
| NIXL GitHub | https://github.com/ai-dynamo/nixl |
| NIXL Docs | https://github.com/ai-dynamo/nixl/blob/main/docs/nixl.md |
| NIXL Python API | https://github.com/ai-dynamo/nixl/blob/main/docs/python_api.md |
| vLLM NIXL Guide | https://docs.vllm.ai/en/stable/features/nixl_connector_usage/ |